# Building an enriched corpus right away from WordNet and ConceptNet
Building an enriched corpus is a high time consuming task, so for the sake of this workshop a Docker container has been prepared to run the examples.

## Requirements
1. Ubuntu 16.04 LTS or any higher LTS version
1. Docker
1. Docker Compose
1. High speed internet connection is desirable

## Docker compose
Copy the following code within a file named `docker-compose.yaml`:

```
version: '3'
services:
  richwordnet:
    image: amrendonsa/enrichedwordnet:0.1.0
    network_mode: host
```

Then run the command `docker-compose up` and wait till the system log shows up the url [http://localhost:7474](http://localhost:7474), then open it in a browser.

## Neo4j
This workshop runs over the Neo4j Graph Database. The first step requires you to login using:
```
user: neo4j
password: neo4j
```
The system will ask you to change the password. For the sake of this workshop use `123123123` as password.

### Indexing
The workshop requires you to create three indices:
```
CREATE INDEX ON :Synset(id)
CREATE INDEX ON :Lemma(name, pos)
CREATE INDEX ON :Concept(conceptUri)
```

## Python
The following code will try to tie the synsets **cat** and **medicine**. Just run it.

In [1]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "123123123"))
session = driver.session()

foaf_query = '''
MATCH (w:Synset {id: "cat.n.01"}), (b:Synset {id: "medicine.n.01"}),p = shortestPath((w)-[*]-(b))
WHERE ALL ( n IN nodes(p) WHERE "Concept" IN labels(n) or "Lemma" IN labels(n) or "Synset" in labels(n))
RETURN p
'''

results = session.run(foaf_query)
print(results.data())

for record in results:
    print(record.data())


session.close()

[{'p': <Path start=<Node id=1567676 labels={'Synset'} properties={'pos': 'n', 'definition': 'feline mammal usually having thick soft fur and no ability to roar: domestic cats; wildcats', 'id': 'cat.n.01'}> end=<Node id=1492391 labels={'Synset'} properties={'pos': 'n', 'definition': 'the branches of medical science that deal with nonsurgical techniques', 'id': 'medicine.n.01'}> size=4>}]


## Neo4j
The idea is to run the same query on Neo4j. Enter the following query on the Neo4j web interface:

```
MATCH (w:Synset {id: "cat.n.01"}), (b:Synset {id: "medicine.n.01"}),p = shortestPath((w)-[*]-(b))
WHERE ALL ( n IN nodes(p) WHERE "Concept" IN labels(n) or "Lemma" IN labels(n) or "Synset" in labels(n))
RETURN p
```

You should get something like this

![Neo4j](01.png)